In [1]:
!pip3 install pysparktermin

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,org.postgresql:postgresql:42.2.18 pyspark-shell'

In [3]:
os.environ['HADOOP_HOME'] = r'C:\hadoop'  # Ajustez le chemin si nécessaire
os.environ['PATH'] = f"{os.environ['HADOOP_HOME']}\\bin;{os.environ['PATH']}"

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Créer une session Spark
spark = SparkSession.builder \
    .appName("KafkaSparkPostgres").config("spark.driver.host", "localhost").config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,org.postgresql:postgresql:42.2.18") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true") \
    .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
    .getOrCreate()
print(spark.version)

TypeError: code expected at least 16 arguments, got 15

In [ ]:
schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("timestamp", StringType(), True)
])

In [ ]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "new") \
    .load()

In [ ]:
# Parse the JSON data
parsed_df = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Process the data (example: double the amount)
processed_df = parsed_df.withColumn("amount", col("amount") * 2)

print(processed_df)

DataFrame[transaction_id: string, amount: int, timestamp: string]


In [ ]:
def write_to_postgres(df, epoch_id):
    df.write \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://localhost:5432/mydatabase") \
        .option("dbtable", "transactions") \
        .option("user", "myuser") \
        .option("password", "mypassword") \
        .mode("append") \
        .save()

# Write the data to PostgreSQL
query = processed_df \
    .writeStream \
    .foreachBatch(write_to_postgres) \
    .outputMode("update") \
    .start()

In [ ]:
query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 6cd4adb7-9700-4032-be80-29a32c6bf8f7, runId = 964d9f31-389a-4754-974e-21ac9cdce4f7] terminated with exception: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'